## Phi-4 RAG کے ساتھ Azure AI Search

یہ نوٹ بک دکھاتی ہے کہ Phi-4-mini اور Phi-4-multimodal کو Retrieval Augmented Generation (RAG) کے لیے Azure AI Search کے ساتھ کیسے استعمال کیا جا سکتا ہے۔ یہ سنگل موڈیلٹی (صرف متن) اور ملٹی موڈیلٹی (متن اور تصویر) دونوں منظرناموں کا احاطہ کرتی ہے۔

**ضروریات:**
*   Azure AI Search ویکٹر انڈیکس (ایک بنانے کے لیے [ان ہدایات](https://learn.microsoft.com/azure/search/search-get-started-portal-import-vectors?tabs=sample-data-storage%2Cmodel-aoai%2Cconnect-data-storage) پر عمل کریں)
*   Azure AI Foundry میں تعینات Phi-4-mini یا Phi-4-multimodal کے اینڈپوائنٹس


In [ ]:
! pip install azure-ai-inference azure-search-documents

### ٹیکسٹ-اونلی RAG کے ساتھ Phi-4-mini

یہ حصہ دکھاتا ہے کہ Phi-4-mini کو RAG کے لیے چیٹ مکمل کرنے والے ماڈل کے طور پر کیسے استعمال کیا جا سکتا ہے، صرف ٹیکسٹ کو ان پٹ کے طور پر استعمال کرتے ہوئے۔ اس میں Azure AI Foundry Inference اور Azure AI Search سے جڑنا، متعلقہ دستاویزات حاصل کرنا، اور حاصل کردہ سیاق و سباق کا استعمال کرتے ہوئے جواب تیار کرنا شامل ہے۔


In [ ]:
# Step 1: Connect to Azure AI Foundry Inference & Azure AI Search
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery

chat_client = ChatCompletionsClient(
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT"], # Phi-4-mini endpoint
    credential=AzureKeyCredential(os.environ["AZURE_INFERENCE_CREDENTIAL"]),
)

search_client = SearchClient(
    endpoint=os.environ["AZURE_SEARCH_ENDPOINT"],
    index_name=os.environ["AZURE_SEARCH_INDEX"],
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_KEY"])
)

# Step 2: Retrieve relevant documents from Azure AI Search
def retrieve_documents(query: str, top: int = 10):
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=top, fields="text_vector")
    results = search_client.search(search_text=query,vector_queries=[vector_query],select=["content"],top=top)
    return [doc["content"] for doc in results]

# Step 3: Generate answer using RAG!
def generate_rag_response(query: str):
    docs = retrieve_documents(query)
    context = "\n---\n".join(docs)
    prompt = f"""You are a helpful assistant. Use only the following context to answer the question. If the answer isn't in the context, say 'I don't know'.
    Context: {context} Question: {query} Answer:"""
    response = chat_client.complete(messages=[UserMessage(content=prompt)])
    return response.choices[0].message.content

# Example usage
user_query = "What is the capital of France?"
answer = generate_rag_response(user_query)
print(f"Q: {user_query}\nA: {answer}")


### Phi-4-multimodal کے ساتھ ملٹی موڈل RAG

یہ حصہ دکھاتا ہے کہ کس طرح Phi-4-multimodal کو RAG کے لیے چیٹ مکمل کرنے کے ماڈل کے طور پر استعمال کیا جا سکتا ہے، جس میں متن اور تصویر دونوں ان پٹ شامل ہیں۔ یہ Azure AI Inference اور Azure AI Search سے جڑنے، متعلقہ دستاویزات حاصل کرنے، اور ایک ملٹی موڈل جواب تیار کرنے کا احاطہ کرتا ہے۔

**نوٹ:** اگر آپ کے Azure AI Search انڈیکس میں `text_vector` اور `image_vector` فیلڈز موجود ہیں تو آپ ملٹی ویکٹر کوئری بھی انجام دے سکتے ہیں۔


In [ ]:
# Step 1: Connect to Azure AI Inference & Azure AI Search
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery


chat_client = ChatCompletionsClient(
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT"], #Phi-4-multimodal serverless endpoint
    credential=AzureKeyCredential(os.environ["AZURE_INFERENCE_CREDENTIAL"]),
)

search_client = SearchClient(
    endpoint=os.environ["AZURE_SEARCH_ENDPOINT"],
    index_name=os.environ["AZURE_SEARCH_INDEX"],
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_KEY"])
)

# Step 2: Retrieve relevant documents from Azure AI Search
def retrieve_documents(query: str, top: int = 3):
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=top, fields="text_vector")
    results = search_client.search(search_text=query, vector_queries=[vector_query], select=["content"], top=top)    
    return [doc["content"] for doc in results]

## Example for Muli-modal search if you have a text_vector AND image_vector field in your vector_index
## NOTE, image vectorization is in preview at the time of writing this code, please use azure-search-documents pypi version >11.6.0b6 
# def retrieve_documents_multimodal(query: str, image_url: str, top: int = 3):
#     text_vector_query = VectorizableTextQuery(
#         text=query,
#         k_nearest_neighbors=top,
#         fields="text_vector",
#         weight=0.5  # Adjust weight as needed
#     )
#     image_vector_query = VectorizableImageUrlQuery(
#         url=image_url,
#         k_nearest_neighbors=top,
#         fields="image_vector",
#         weight=0.5  # Adjust weight as needed
#     )

#     results = search_client.search(
#         search_text=query,  
#         vector_queries=[text_vector_query, image_vector_query],
#         select=["content"],
#         top=top
#     )
#     return [doc["content"] for doc in results]


# Step 3: Generate a multimodal RAG-based answer using retrieved text and an image input
def generate_multimodal_rag_response(query: str, image_url: str):
    # Retrieve text context from search
    docs = retrieve_documents(query)
    context = "\n---\n".join(docs)

    # Build a prompt that combines the retrieved context with the user query
    prompt = f"""You are a helpful assistant. Use only the following context to answer the question. If the answer isn't in the context, say 'I don't know'.
    Context: {context} Question: {query} Answer:"""
    # Create a chat request that includes both text and image input
    response = chat_client.complete(
        messages=[
            SystemMessage(content="You are a helpful assistant that can process both text and images."),
            UserMessage(
                content=[
                    TextContentItem(text=prompt),
                    ImageContentItem(image_url=ImageUrl(url=image_url, detail=ImageDetailLevel.HIGH)),
                ]
            ),
        ]
    )
    return response.choices[0].message.content

# Example usage
user_query = "Can you search for similar items to this shoe?"
sample_image_url = "https://images.unsplash.com/photo-1542291026-7eec264c27ff?q=80&w=1770&auto=format&fit=crop&ixlib=rb-4.0.3"
answer = generate_multimodal_rag_response(user_query, sample_image_url)
print(f"Q: {user_query}\nA: {answer}")


---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا غیر درستیاں ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ ہم اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے ذمہ دار نہیں ہیں۔
